In [24]:
import numpy as np
import pandas as pd
import re
import nltk
import spacy
import string
pd.options.mode.chained_assignment = None



In [25]:
full_df = pd.read_csv("/content/sample.csv", nrows=5000)
df = full_df[["text"]]
df["text"] = df["text"].astype(str)
full_df.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,119237,105834,True,Wed Oct 11 06:55:44 +0000 2017,@AppleSupport causing the reply to be disregar...,119236,NaN
1,119238,ChaseSupport,False,Wed Oct 11 13:25:49 +0000 2017,@105835 Your business means a lot to us. Pleas...,NaN,119239.0
2,119239,105835,True,Wed Oct 11 13:00:09 +0000 2017,@76328 I really hope you all change but I'm su...,119238,NaN
3,119240,VirginTrains,False,Tue Oct 10 15:16:08 +0000 2017,@105836 LiveChat is online at the moment - htt...,119241,119242.0
4,119241,105836,True,Tue Oct 10 15:17:21 +0000 2017,@VirginTrains see attached error message. I've...,119243,119240.0


Lower Casing

In [26]:
df["text_lower"] = df["text"].str.lower()
df.head()

,text,text_lower
0,@AppleSupport causing the reply to be disregar...,@applesupport causing the reply to be disregar...
1,@105835 Your business means a lot to us. Pleas...,@105835 your business means a lot to us. pleas...
2,@76328 I really hope you all change but I'm su...,@76328 i really hope you all change but i'm su...
3,@105836 LiveChat is online at the moment - htt...,@105836 livechat is online at the moment - htt...
4,@VirginTrains see attached error message. I've...,@virgintrains see attached error message. i've...


Removal of Punctuations

In [27]:
df.drop(["text_lower"], axis=1, inplace=True)

PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

df["text_wo_punct"] = df["text"].apply(lambda text: remove_punctuation(text))
df.head()

,text,text_wo_punct
0,@AppleSupport causing the reply to be disregar...,AppleSupport causing the reply to be disregard...
1,@105835 Your business means a lot to us. Pleas...,105835 Your business means a lot to us Please ...
2,@76328 I really hope you all change but I'm su...,76328 I really hope you all change but Im sure...
3,@105836 LiveChat is online at the moment - htt...,105836 LiveChat is online at the moment https...
4,@VirginTrains see attached error message. I've...,VirginTrains see attached error message Ive tr...


Removal of stopwords

In [28]:
from nltk.corpus import stopwords
nltk.download('stopwords')

", ".join(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


"a, about, above, after, again, against, ain, all, am, an, and, any, are, aren, aren't, as, at, be, because, been, before, being, below, between, both, but, by, can, couldn, couldn't, d, did, didn, didn't, do, does, doesn, doesn't, doing, don, don't, down, during, each, few, for, from, further, had, hadn, hadn't, has, hasn, hasn't, have, haven, haven't, having, he, he'd, he'll, her, here, hers, herself, he's, him, himself, his, how, i, i'd, if, i'll, i'm, in, into, is, isn, isn't, it, it'd, it'll, it's, its, itself, i've, just, ll, m, ma, me, mightn, mightn't, more, most, mustn, mustn't, my, myself, needn, needn't, no, nor, not, now, o, of, off, on, once, only, or, other, our, ours, ourselves, out, over, own, re, s, same, shan, shan't, she, she'd, she'll, she's, should, shouldn, shouldn't, should've, so, some, such, t, than, that, that'll, the, their, theirs, them, themselves, then, there, these, they, they'd, they'll, they're, they've, this, those, through, to, too, under, until, up, 

In [29]:
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):

    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

df["text_wo_stop"] = df["text_wo_punct"].apply(lambda text: remove_stopwords(text))
df.head()

,text,text_wo_punct,text_wo_stop
0,@AppleSupport causing the reply to be disregar...,AppleSupport causing the reply to be disregard...,AppleSupport causing reply disregarded tapped ...
1,@105835 Your business means a lot to us. Pleas...,105835 Your business means a lot to us Please ...,105835 Your business means lot us Please DM na...
2,@76328 I really hope you all change but I'm su...,76328 I really hope you all change but Im sure...,76328 I really hope change Im sure wont Becaus...
3,@105836 LiveChat is online at the moment - htt...,105836 LiveChat is online at the moment https...,105836 LiveChat online moment httpstcoSY94VtU8...
4,@VirginTrains see attached error message. I've...,VirginTrains see attached error message Ive tr...,VirginTrains see attached error message Ive tr...


Removal of Frequent words

In [30]:
from collections import Counter
cnt = Counter()
for text in df["text_wo_stop"].values:
    for word in text.split():
        cnt[word] += 1

cnt.most_common(10)

[('I', 34),
 ('us', 25),
 ('DM', 19),
 ('help', 17),
 ('httpstcoGDrqU22YpT', 12),
 ('AppleSupport', 11),
 ('Thanks', 11),
 ('phone', 9),
 ('Hi', 8),
 ('get', 8)]

In [31]:
FREQWORDS = set([w for (w, wc) in cnt.most_common(10)])
def remove_freqwords(text):
    """custom function to remove the frequent words"""
    return " ".join([word for word in str(text).split() if word not in FREQWORDS])

df["text_wo_stopfreq"] = df["text_wo_stop"].apply(lambda text: remove_freqwords(text))
df.head()

,text,text_wo_punct,text_wo_stop,text_wo_stopfreq
0,@AppleSupport causing the reply to be disregar...,AppleSupport causing the reply to be disregard...,AppleSupport causing reply disregarded tapped ...,causing reply disregarded tapped notification ...
1,@105835 Your business means a lot to us. Pleas...,105835 Your business means a lot to us Please ...,105835 Your business means lot us Please DM na...,105835 Your business means lot Please name zip...
2,@76328 I really hope you all change but I'm su...,76328 I really hope you all change but Im sure...,76328 I really hope change Im sure wont Becaus...,76328 really hope change Im sure wont Because ...
3,@105836 LiveChat is online at the moment - htt...,105836 LiveChat is online at the moment https...,105836 LiveChat online moment httpstcoSY94VtU8...,105836 LiveChat online moment httpstcoSY94VtU8...
4,@VirginTrains see attached error message. I've...,VirginTrains see attached error message Ive tr...,VirginTrains see attached error message Ive tr...,VirginTrains see attached error message Ive tr...


Removal of Rare words

In [32]:
df.drop(["text_wo_punct", "text_wo_stop"], axis=1, inplace=True)

n_rare_words = 10
RAREWORDS = set([w for (w, wc) in cnt.most_common()[:-n_rare_words-1:-1]])
def remove_rarewords(text):
    """custom function to remove the rare words"""
    return " ".join([word for word in str(text).split() if word not in RAREWORDS])

df["text_wo_stopfreqrare"] = df["text_wo_stopfreq"].apply(lambda text: remove_rarewords(text))
df.head()

,text,text_wo_stopfreq,text_wo_stopfreqrare
0,@AppleSupport causing the reply to be disregar...,causing reply disregarded tapped notification ...,causing reply disregarded tapped notification ...
1,@105835 Your business means a lot to us. Pleas...,105835 Your business means lot Please name zip...,105835 Your business means lot Please name zip...
2,@76328 I really hope you all change but I'm su...,76328 really hope change Im sure wont Because ...,76328 really hope change Im sure wont Because ...
3,@105836 LiveChat is online at the moment - htt...,105836 LiveChat online moment httpstcoSY94VtU8...,105836 LiveChat online moment httpstcoSY94VtU8...
4,@VirginTrains see attached error message. I've...,VirginTrains see attached error message Ive tr...,VirginTrains see attached error message Ive tr...


STEMMING

In [33]:
from nltk.stem.porter import PorterStemmer


df.drop(["text_wo_stopfreq", "text_wo_stopfreqrare"], axis=1, inplace=True)

stemmer = PorterStemmer()
def stem_words(text):
    return " ".join([stemmer.stem(word) for word in text.split()])

df["text_stemmed"] = df["text"].apply(lambda text: stem_words(text))
df.head()

,text,text_stemmed
0,@AppleSupport causing the reply to be disregar...,@applesupport caus the repli to be disregard a...
1,@105835 Your business means a lot to us. Pleas...,@105835 your busi mean a lot to us. pleas dm y...
2,@76328 I really hope you all change but I'm su...,@76328 i realli hope you all chang but i'm sur...
3,@105836 LiveChat is online at the moment - htt...,@105836 livechat is onlin at the moment - http...
4,@VirginTrains see attached error message. I've...,@virgintrain see attach error message. i'v tri...


In [34]:
from nltk.stem.snowball import SnowballStemmer
SnowballStemmer.languages

('arabic',
 'danish',
 'dutch',
 'english',
 'finnish',
 'french',
 'german',
 'hungarian',
 'italian',
 'norwegian',
 'porter',
 'portuguese',
 'romanian',
 'russian',
 'spanish',
 'swedish')

Lemmatization

In [37]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

df["text_lemmatized"] = df["text"].apply(lambda text: lemmatize_words(text))
df.head()

[nltk_data] Downloading package wordnet to /root/nltk_data...


,text,text_stemmed,text_lemmatized
0,@AppleSupport causing the reply to be disregar...,@applesupport caus the repli to be disregard a...,@AppleSupport causing the reply to be disregar...
1,@105835 Your business means a lot to us. Pleas...,@105835 your busi mean a lot to us. pleas dm y...,@105835 Your business mean a lot to us. Please...
2,@76328 I really hope you all change but I'm su...,@76328 i realli hope you all chang but i'm sur...,@76328 I really hope you all change but I'm su...
3,@105836 LiveChat is online at the moment - htt...,@105836 livechat is onlin at the moment - http...,@105836 LiveChat is online at the moment - htt...
4,@VirginTrains see attached error message. I've...,@virgintrain see attach error message. i'v tri...,@VirginTrains see attached error message. I've...


In [38]:
lemmatizer.lemmatize("running")

'running'

In [39]:
lemmatizer.lemmatize("running", "v")

'run'

In [40]:
print("Word is : stripes")
print("Lemma result for verb : ",lemmatizer.lemmatize("stripes", 'v'))
print("Lemma result for noun : ",lemmatizer.lemmatize("stripes", 'n'))

Word is : stripes
Lemma result for verb :  strip
Lemma result for noun :  stripe


In [42]:
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
nltk.download('averaged_perceptron_tagger_eng')

lemmatizer = WordNetLemmatizer()
wordnet_map = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV}
def lemmatize_words(text):
    pos_tagged_text = nltk.pos_tag(text.split())
    return " ".join([lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tagged_text])

df["text_lemmatized"] = df["text"].apply(lambda text: lemmatize_words(text))
df.head()

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


,text,text_stemmed,text_lemmatized
0,@AppleSupport causing the reply to be disregar...,@applesupport caus the repli to be disregard a...,@AppleSupport cause the reply to be disregard ...
1,@105835 Your business means a lot to us. Pleas...,@105835 your busi mean a lot to us. pleas dm y...,@105835 Your business mean a lot to us. Please...
2,@76328 I really hope you all change but I'm su...,@76328 i realli hope you all chang but i'm sur...,@76328 I really hope you all change but I'm su...
3,@105836 LiveChat is online at the moment - htt...,@105836 livechat is onlin at the moment - http...,@105836 LiveChat be online at the moment - htt...
4,@VirginTrains see attached error message. I've...,@virgintrain see attach error message. i'v tri...,@VirginTrains see attached error message. I've...


Removal of Emoticons

In [45]:
import re

EMOTICONS = [
    ":-)", ":)", ":-(", ":(", ";-)", ";)",
    ":-D", ":D", ":-P", ":P"
]

def remove_emoticons(text):

    emoticon_pattern = re.compile(u'(' + u'|'.join(re.escape(k) for k in EMOTICONS) + u')')
    return emoticon_pattern.sub(r'', text)

print(remove_emoticons("Hello :-) How are you :D"))



Hello  How are you 


In [46]:
remove_emoticons("I am sad :(")

'I am sad '

Removal of URLs

In [47]:
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

In [48]:
text = "Driverless AI NLP blog post on https://www.h2o.ai/blog/detecting-sarcasm-is-difficult-but-ai-may-have-an-answer/"
remove_urls(text)

'Driverless AI NLP blog post on '

In [49]:
text = "Please refer to link http://lnkd.in/ecnt5yC for the paper"
remove_urls(text)

'Please refer to link  for the paper'

In [50]:
text = "Want to know more. Checkout www.h2o.ai for additional information"
remove_urls(text)

'Want to know more. Checkout  for additional information'

In [51]:
def remove_html(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)

text = """<div>
<h1> H2O</h1>
<p> AutoML</p>
<a href="https://www.h2o.ai/products/h2o-driverless-ai/"> Driverless AI</a>
</div>"""

print(remove_html(text))


 H2O
 AutoML
 Driverless AI



Conversion of Emoji to Words

In [57]:
import re

UNICODE_EMO = {
    "🔥": "fire",
    "😂": "laughing",
    "❤️": "love",
    "😢": "crying",
    "👍": "thumbs up"
}

def convert_emojis(text):
    for emot in UNICODE_EMO:
        text = re.sub(
            r'(' + re.escape(emot) + r')',
            "_".join(
                UNICODE_EMO[emot]
                .replace(",", "")
                .replace(":", "")
                .split()
            ),
            text
        )
    return text

text = "game is on 🔥 😂"
print(convert_emojis(text))


game is on fire laughing


In [58]:
text = "Hilarious 😂"
convert_emojis(text)

'Hilarious laughing'

Spelling Correction

In [53]:
!pip install pyspellchecker


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 88.3 MB/s eta 0:00:00


In [54]:
from spellchecker import SpellChecker

spell = SpellChecker()
def correct_spellings(text):
    corrected_text = []
    misspelled_words = spell.unknown(text.split())
    for word in text.split():
        if word in misspelled_words:
            corrected_text.append(spell.correction(word))
        else:
            corrected_text.append(word)
    return " ".join(corrected_text)

text = "speling correctin"
correct_spellings(text)

'spelling correcting'

In [55]:
text = "thnks for readin the notebook"
correct_spellings(text)

'thanks for reading the notebook'